In [172]:
from get_lineups import get_lineups
from nba_api.stats.endpoints import PlayByPlayV3
from nba_api.stats.endpoints import boxscoretraditionalv2
import pandas as pd
import numpy as np
import re

In [157]:
game_id = "0022400062"

In [178]:
TIP_TO_RE = re.compile(r"Tip to ([A-Za-z .'-]+)")

In [176]:
boxscore = boxscoretraditionalv2.BoxScoreTraditionalV2(game_id=game_id).get_data_frames()[0]
boxscore['LAST_NAME'] = boxscore.apply(lambda row: row['PLAYER_NAME'].replace(row['NICKNAME'], '').strip(), axis = 1)
player_team_dict = {}
for idx, player in boxscore.iterrows():
    if player['MIN'] == None:
        continue

    last_name = player['LAST_NAME']
    team = player['TEAM_ABBREVIATION']
    player_team_dict[last_name] = team

player_team_dict

{'McDaniels': 'MIN',
 'Randle': 'MIN',
 'Gobert': 'MIN',
 'Edwards': 'MIN',
 'Conley': 'MIN',
 'Reid': 'MIN',
 'DiVincenzo': 'MIN',
 'Alexander-Walker': 'MIN',
 'Ingles': 'MIN',
 'Hachimura': 'LAL',
 'James': 'LAL',
 'Davis': 'LAL',
 'Reaves': 'LAL',
 'Russell': 'LAL',
 'Christie': 'LAL',
 'Vincent': 'LAL',
 'Hayes': 'LAL',
 'Knecht': 'LAL'}

In [158]:
pbp = PlayByPlayV3(game_id=game_id).get_data_frames()[0]

In [159]:
pbp['time_in_period'] = (12 * 60 - (60 * pbp['clock'].str[2:4].astype(int) + pbp['clock'].str[5:10].astype(float))) * 10
pbp['time'] = (pbp['period'].astype(int) - 1) * 60 * 12 * 10 + pbp['time_in_period']  # solve: OT
pbp[['scoreHome', 'scoreAway']] = pbp[['scoreHome', 'scoreAway']].replace('', np.nan)
pbp[['scoreHome', 'scoreAway']] = pbp[['scoreHome', 'scoreAway']].ffill()

In [160]:
plays = pbp[['actionNumber', 'teamId', 'scoreHome', 'scoreAway', 'description', 'actionType', 'subType', 'time', 'location']].copy()
plays['possessionEnd'] = False

In [ ]:
for idx, row in plays.iterrows():
    actionType = row['actionType']
    subType = row['subType']
    actionNumber = row['actionNumber']
    location = row['location']
    description = row['description']

    if actionType in ('Made Shot', 'Turnover'):
        plays.at[idx, 'possessionEnd'] = True

    elif actionType == 'period':
        if subType == 'start':
            plays.at[idx, 'possessionEnd'] = True

    elif actionType == 'Missed Shot':

        next_idx = idx + 1
        while plays.at[next_idx, 'actionNumber'] == actionNumber:
            next_idx += 1
        next_play = plays.loc[next_idx]

        assert next_play['actionType'] == 'Rebound', f"Next play {next_play['description']} (action {next_play['actionNumber']}) (row {idx}) after {row['description']} is not rebound"
        if location != next_play['location']:
            plays.at[next_idx, 'possessionEnd'] = True

    elif actionType == 'Free Throw':

        next_idx = idx + 1
        while plays.at[next_idx, 'actionNumber'] == actionNumber:
            next_idx += 1
        next_play = plays.loc[next_idx]

        if next_play['actionType'] == 'Rebound':
            if location != next_play['location']:
                plays.at[next_idx, 'possessionEnd'] = True

        else:
            plays.at[next_idx, 'possessionEnd'] = True

    elif actionType == 'Jump Ball':
        m = TIP_TO_RE.search(description)
        assert m, f"No player found in description: {description} (row {idx})"
        player = m.group(1).strip()
        # find out which team got jump ball using dict
        # add possession column with team name of possession using abbreviation in dict

In [169]:
plays

,actionNumber,teamId,scoreHome,scoreAway,description,actionType,subType,time,location,possessionEnd
0,2,0,0,0,Start of 1st Period (10:07 PM EST),period,start,0.0,,True
1,4,1610612747,0,0,Jump Ball Davis vs. Gobert: Tip to Conley,Jump Ball,,0.0,h,False
2,7,1610612750,0,0,MISS Edwards 27' 3PT Jump Shot,Missed Shot,Jump Shot,280.0,v,False
3,8,1610612747,0,0,L. James REBOUND (Off:0 Def:1),Rebound,Unknown,310.0,h,True
4,9,1610612747,3,0,Hachimura 26' 3PT Jump Shot (3 PTS) (Davis 1 AST),Made Shot,Jump Shot,420.0,h,True
...,...,...,...,...,...,...,...,...,...,...
494,675,1610612747,110,103,L. James Out of Bounds - Bad Pass Turnover Tur...,Turnover,Out of Bounds - Bad Pass Turnover,28567.0,h,True
495,676,1610612750,110,103,MISS Randle 3' Driving Layup,Missed Shot,Driving Layup Shot,28604.0,v,False
496,676,1610612747,110,103,L. James BLOCK (2 BLK),,,28604.0,h,False
497,678,1610612747,110,103,L. James REBOUND (Off:0 Def:5),Rebound,Unknown,28630.0,h,True


In [163]:
plays['actionType'].unique()

array(['period', 'Jump Ball', 'Missed Shot', 'Rebound', 'Made Shot',
       'Foul', '', 'Violation', 'Turnover', 'Timeout', 'Substitution',
       'Free Throw'], dtype=object)

In [164]:
plays[340:370]

,actionNumber,teamId,scoreHome,scoreAway,description,actionType,subType,time,location,possessionEnd
340,462,1610612747,76,65,SUB: Hayes FOR Reaves,Substitution,,19800.0,h,False
341,463,1610612747,76,65,SUB: L. James FOR Hachimura,Substitution,,19800.0,h,False
342,474,1610612750,76,65,SUB: Randle FOR Gobert,Substitution,,19800.0,v,False
343,466,1610612747,77,65,Davis Free Throw 1 of 1 (22 PTS),Free Throw,Free Throw 1 of 1,19800.0,h,False
344,467,1610612750,77,65,MISS Randle 13' Jump Shot,Missed Shot,Jump Shot,19970.0,v,True
345,468,1610612747,77,65,Davis REBOUND (Off:2 Def:9),Rebound,Unknown,20060.0,h,True
346,469,1610612747,77,65,MISS Knecht 26' 3PT Running Jump Shot,Missed Shot,Running Jump Shot,20080.0,h,False
347,470,1610612750,77,65,Randle REBOUND (Off:0 Def:5),Rebound,Unknown,20100.0,v,True
348,471,1610612747,77,65,Vincent P.FOUL (P1.PN) (J.Orr),Foul,Personal,20100.0,h,False
349,473,0,77,65,Timberwolves Timeout: Regular (Reg.3 Short 0),Timeout,Regular,20100.0,v,False


In [165]:
pbp.head()

,gameId,actionNumber,clock,period,teamId,teamTricode,personId,playerName,playerNameI,xLegacy,...,pointsTotal,location,description,actionType,subType,videoAvailable,shotValue,actionId,time_in_period,time
0,0022400062,2,PT12M00.00S,1,0,,0,,,0,...,0,,Start of 1st Period (10:07 PM EST),period,start,0,0,1,0.0,0.0
1,0022400062,4,PT12M00.00S,1,1610612747,LAL,203076,Davis,A. Davis,0,...,0,h,Jump Ball Davis vs. Gobert: Tip to Conley,Jump Ball,,1,0,2,0.0,0.0
2,0022400062,7,PT11M32.00S,1,1610612750,MIN,1630162,Edwards,A. Edwards,-89,...,0,v,MISS Edwards 27' 3PT Jump Shot,Missed Shot,Jump Shot,1,3,3,280.0,280.0
3,0022400062,8,PT11M29.00S,1,1610612747,LAL,2544,James,L. James,0,...,0,h,L. James REBOUND (Off:0 Def:1),Rebound,Unknown,1,0,4,310.0,310.0
4,0022400062,9,PT11M18.00S,1,1610612747,LAL,1629060,Hachimura,R. Hachimura,121,...,3,h,Hachimura 26' 3PT Jump Shot (3 PTS) (Davis 1 AST),Made Shot,Jump Shot,1,3,5,420.0,420.0


In [166]:
lineups = get_lineups(game_id=game_id)
lineups

c:\Users\svest\Desktop\NBA-Adjusted-Plus-Minus\get_lineups.py:55: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  lineup = pd.concat(


,Start_Time,End_Time,Player_1_Home_ID,Player_1_Home_Name,Player_2_Home_ID,Player_2_Home_Name,Player_3_Home_ID,Player_3_Home_Name,Player_4_Home_ID,Player_4_Home_Name,...,Player_1_Away_ID,Player_1_Away_Name,Player_2_Away_ID,Player_2_Away_Name,Player_3_Away_ID,Player_3_Away_Name,Player_4_Away_ID,Player_4_Away_Name,Player_5_Away_ID,Player_5_Away_Name
0,0.0,3160.0,201144,Mike Conley,203497,Rudy Gobert,203944,Julius Randle,1630162,Anthony Edwards,...,2544,LeBron James,203076,Anthony Davis,1626156,D'Angelo Russell,1629060,Rui Hachimura,1630559,Austin Reaves
1,3160.0,3540.0,201144,Mike Conley,203497,Rudy Gobert,203944,Julius Randle,1630162,Anthony Edwards,...,203076,Anthony Davis,1626156,D'Angelo Russell,1629060,Rui Hachimura,1630559,Austin Reaves,1631108,Max Christie
2,3540.0,4110.0,203497,Rudy Gobert,1630162,Anthony Edwards,1630183,Jaden McDaniels,203076,Anthony Davis,...,1629060,Rui Hachimura,1630559,Austin Reaves,1631108,Max Christie,1628978,Donte DiVincenzo,1629675,Naz Reid
3,4110.0,5370.0,203497,Rudy Gobert,1630162,Anthony Edwards,1630183,Jaden McDaniels,203076,Anthony Davis,...,1630559,Austin Reaves,1631108,Max Christie,1628978,Donte DiVincenzo,1629675,Naz Reid,1629216,Gabe Vincent
4,5370.0,5880.0,203497,Rudy Gobert,1630162,Anthony Edwards,203076,Anthony Davis,1631108,Max Christie,...,1629675,Naz Reid,1629216,Gabe Vincent,1629638,Nickeil Alexander-Walker,2544,LeBron James,1629637,Jaxson Hayes
5,5880.0,6520.0,1630162,Anthony Edwards,203076,Anthony Davis,1631108,Max Christie,1628978,Donte DiVincenzo,...,1629216,Gabe Vincent,1629638,Nickeil Alexander-Walker,2544,LeBron James,1629637,Jaxson Hayes,203944,Julius Randle
6,6520.0,7200.0,1630162,Anthony Edwards,1631108,Max Christie,1628978,Donte DiVincenzo,1629675,Naz Reid,...,1629638,Nickeil Alexander-Walker,2544,LeBron James,1629637,Jaxson Hayes,203944,Julius Randle,1626156,D'Angelo Russell
7,7200.0,9270.0,1628978,Donte DiVincenzo,1629675,Naz Reid,1629638,Nickeil Alexander-Walker,2544,LeBron James,...,203944,Julius Randle,1626156,D'Angelo Russell,201144,Mike Conley,1629060,Rui Hachimura,1642261,Dalton Knecht
8,9270.0,9860.0,1628978,Donte DiVincenzo,1629675,Naz Reid,2544,LeBron James,1629637,Jaxson Hayes,...,201144,Mike Conley,1629060,Rui Hachimura,1642261,Dalton Knecht,203497,Rudy Gobert,1630183,Jaden McDaniels
9,9860.0,9960.0,1628978,Donte DiVincenzo,1629675,Naz Reid,1626156,D'Angelo Russell,201144,Mike Conley,...,1642261,Dalton Knecht,203497,Rudy Gobert,1630183,Jaden McDaniels,203076,Anthony Davis,1630559,Austin Reaves


In [167]:
df = pd.merge_asof(lineups, pbp[['scoreHome', 'scoreAway', 'time']], left_on='End_Time', right_on='time', direction='backward')

In [168]:
df

,Start_Time,End_Time,Player_1_Home_ID,Player_1_Home_Name,Player_2_Home_ID,Player_2_Home_Name,Player_3_Home_ID,Player_3_Home_Name,Player_4_Home_ID,Player_4_Home_Name,...,Player_2_Away_Name,Player_3_Away_ID,Player_3_Away_Name,Player_4_Away_ID,Player_4_Away_Name,Player_5_Away_ID,Player_5_Away_Name,scoreHome,scoreAway,time
0,0.0,3160.0,201144,Mike Conley,203497,Rudy Gobert,203944,Julius Randle,1630162,Anthony Edwards,...,Anthony Davis,1626156,D'Angelo Russell,1629060,Rui Hachimura,1630559,Austin Reaves,11,7,3160.0
1,3160.0,3540.0,201144,Mike Conley,203497,Rudy Gobert,203944,Julius Randle,1630162,Anthony Edwards,...,D'Angelo Russell,1629060,Rui Hachimura,1630559,Austin Reaves,1631108,Max Christie,13,7,3540.0
2,3540.0,4110.0,203497,Rudy Gobert,1630162,Anthony Edwards,1630183,Jaden McDaniels,203076,Anthony Davis,...,Austin Reaves,1631108,Max Christie,1628978,Donte DiVincenzo,1629675,Naz Reid,16,10,4110.0
3,4110.0,5370.0,203497,Rudy Gobert,1630162,Anthony Edwards,1630183,Jaden McDaniels,203076,Anthony Davis,...,Max Christie,1628978,Donte DiVincenzo,1629675,Naz Reid,1629216,Gabe Vincent,18,13,5370.0
4,5370.0,5880.0,203497,Rudy Gobert,1630162,Anthony Edwards,203076,Anthony Davis,1631108,Max Christie,...,Gabe Vincent,1629638,Nickeil Alexander-Walker,2544,LeBron James,1629637,Jaxson Hayes,20,15,5880.0
5,5880.0,6520.0,1630162,Anthony Edwards,203076,Anthony Davis,1631108,Max Christie,1628978,Donte DiVincenzo,...,Nickeil Alexander-Walker,2544,LeBron James,1629637,Jaxson Hayes,203944,Julius Randle,20,18,6520.0
6,6520.0,7200.0,1630162,Anthony Edwards,1631108,Max Christie,1628978,Donte DiVincenzo,1629675,Naz Reid,...,LeBron James,1629637,Jaxson Hayes,203944,Julius Randle,1626156,D'Angelo Russell,22,23,7200.0
7,7200.0,9270.0,1628978,Donte DiVincenzo,1629675,Naz Reid,1629638,Nickeil Alexander-Walker,2544,LeBron James,...,D'Angelo Russell,201144,Mike Conley,1629060,Rui Hachimura,1642261,Dalton Knecht,34,27,9270.0
8,9270.0,9860.0,1628978,Donte DiVincenzo,1629675,Naz Reid,2544,LeBron James,1629637,Jaxson Hayes,...,Rui Hachimura,1642261,Dalton Knecht,203497,Rudy Gobert,1630183,Jaden McDaniels,37,29,9860.0
9,9860.0,9960.0,1628978,Donte DiVincenzo,1629675,Naz Reid,1626156,D'Angelo Russell,201144,Mike Conley,...,Rudy Gobert,1630183,Jaden McDaniels,203076,Anthony Davis,1630559,Austin Reaves,39,29,9960.0
